In [12]:
from matrix import *
from random import random
#implementation of Base strassen matrix multiplication
def isPwr2(x): 
    #uses the fact that a power of 2 in binary has one 1 and the remaining digits are 0
    #so 16 = 10000, 32 = 100000 and so on 
    #and eg. 16 - 1 = 15 = 01111 this holds for every 2**n
    #so bitwise 16 && 15 = 1000 && 0111 = 0000
    #so not(16 && 15) returns 1
    return not(x & (x - 1))

def strassen_matrix_mul(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError("Wrong matrix shape: number of columns of A is %d, number of rows of B is %d"
                         %(A.num_of_cols, B.num_of_cols) )
    if (A.num_of_cols != A.num_of_rows or B.num_of_cols != B.num_of_rows) and not (isPwr2(A.num_of_cols)) :
        raise NotImplementedError("This implemetation deals with SQUARE matrices products with use instead GENERAL_strassen_matrix_mul")
    
    #Base case
    if A.num_of_cols < 32:
        return gauss_matrix_mult(A,B)
    
    #quadrant subdivision
    n_half = A.num_of_cols
    A11 = A.submatrix(0, n_half, n_half, n_half)

In [26]:
nc = 10
nr = 10

A = Matrix([[random() for j in range(nc)] for i in range(nr)])
B = Matrix([[random() for j in range(nc)] for i in range(nr)])

In [27]:
strassen_matrix_mul(A,B)

In [28]:
n_half = A.num_of_cols
A11 = A.submatrix(0, n_half, n_half, n_half)

In [29]:
A11

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]